# Classify organs present in ultrasound images

In [ ]:
import os
import numpy
from fastai.vision import *

In [ ]:
# Indicate the location of the pretrained torchvision models 
os.environ['TORCH_HOME'] = Path('models/resnet').absolute().as_posix()

## Data import

In [ ]:
valid_frac = 0.2
data_path = "data/Output_organ/"
data = ImageDataBunch.from_folder(data_path, valid_pct=valid_frac)
#data = ImageDataBunch.from_folder(data_path, valid_pct=valid_frac,tfms=get_transforms())

## Model initial training

In [ ]:
learn = cnn_learner(data, models.resnet34, metrics=accuracy)
#learn = cnn_learner(data, models.resnet34, metrics=[accuracy, auc_roc_score])

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
lr = 3e-3 #get lr from plot

In [ ]:
callbacks_opt = [callbacks.SaveModelCallback(learn,
                                             monitor='accuracy',
                                             every='improvement',
                                             name='best_accuracy_resnet34'),
                callbacks.EarlyStoppingCallback(learn, monitor='accuracy', min_delta=.01, patience=3)]

In [ ]:
learn.fit(10, lr, callbacks=callbacks_opt)

We select the best model from the early stopping

In [ ]:
learn.load('best_accuracy_resnet34');

In [ ]:
learn.summary()

### Check the confusion matrix to identify problematic classes

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_confusion_matrix(normalize=False, figsize=(14, 8))

## Fine tuning of the model

In [ ]:
learn.unfreeze()

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
callbacks_opt = [callbacks.SaveModelCallback(learn,
                                             monitor='accuracy',
                                             every='improvement',
                                             name='best_accuracy_unfreeze_resnet34'),
                callbacks.EarlyStoppingCallback(learn, monitor='accuracy', min_delta=.003, patience=3)]

In [ ]:
lr_max = 1e-4

In [ ]:
learn.fit(10, slice(lr_max/5, lr_max), callbacks=callbacks_opt)

In [ ]:
interp_02 = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp_02.plot_confusion_matrix(normalize=False, figsize=(14, 8))

In [ ]:
interp_02.plot_top_losses(9)

In [ ]:
interp_02.plot_multi_top_losses()

In [ ]:
#freeze bottom layer
learn.freeze()

In [ ]:
learn.summary()

## data labelling for binary classification

In [ ]:
#valid_frac2 = 0.2
data_path2 = "data/Organ_Liver_Others/"

In [ ]:
#customize label    
data2 = ImageDataBunch.from_folder(data_path2, valid_pct=valid_frac, size =224)).normalize(imagenet_stats)
#data2 = ImageDataBunch.from_folder(data_path2, valid_pct=valid_frac, size =224, ds_tfms=get_transforms()).normalize(imagenet_stats)

## binary classification model

In [ ]:
learn11 = learn.load('best_accuracy_unfreeze_resnet34')

In [ ]:
print (learn11.model[0])

In [ ]:
print (learn11.model[1])

In [ ]:
learn2 = cnn_learner(data2, models.resnet34, metrics=AUROC())
#If "AUROC()" is the wrong statement, change it to "auc_roc_score"

In [ ]:
learn2.model[0].load_state_dict(learn11.model[0].state_dict())

In [ ]:
learn2.lr_find() 
learn2.recorder.plot()   

In [ ]:
lr = 1e-2

In [ ]:
callbacks_opt = [callbacks.SaveModelCallback(learn2,
                                             monitor='AUROC()',
                                             every='improvement',
                                             name='best_AUROC_binary_resnet34'),
                callbacks.EarlyStoppingCallback(learn2, monitor='AUROC()', min_delta=.01, patience=3)]
#If "AUROC()" is the wrong statement, change it to "auc_roc_score"

In [ ]:
learn2.fit(10, lr, callbacks=callbacks_opt)

In [ ]:
learn2.summary()

In [ ]:
interp3 = ClassificationInterpretation.from_learner(learn2)

In [ ]:
interp3.plot_confusion_matrix(normalize=False, figsize=(8, 8))

In [ ]:
interp3.plot_top_losses(9)

In [ ]:
interp3.plot_multi_top_losses()

## prediction

In [ ]:
#img = learn2.data.test_ds[0][0]
img = Image.open(path)

In [ ]:
img 

In [ ]:
learn2.predict(img)